Intelligent User Interfaces: Assignment 1
=========================================


Problem 1
---------

Named Entity Recognition showing tokenization, parts of speech tagging followed by named entity recognition for

> Steve Jobs was an American entrepreneur and business magnate.
> He was the chairman, chief executive officer (CEO), and a co-founder of Apple Inc.,
> chairman and majority shareholder of Pixar, a member of The Walt Disney Company's
> board of directors following its acquisition of Pixar, and the founder, chairman, and CEO of NeXT.
> Jobs is widely recognized as a pioneer of the microcomputer
> revolution of the 1970s and 1980s, along with Apple co-founder Steve Wozniak.


Problem 2
---------

Extract all bigrams, trigrams using ngrams of nltk library

> Machine learning is a necessary field in today's world.
> Data science can do wonders
> Natural Language Processing is how machines understand text


Problem 3
---------

Sentiment analysis using Vader. Print polarity scores for each token along
with compound scores for each sentence. Based on the compound scores,
decide sentiment as positive (if >=0.05), negative (if<+0.05) or neutral otherwise.

Sentences:

- We are happy!
- Today I am Happy
- The best life ever
- I am sad
- We are sad
- We are super sad
- We are all so sad today


Problem 4
---------

Text Summarization of a Wikipedia article

https://en.wikipedia.org/wiki/Artificial_intelligence

1. Data collection from Wikipedia using web scraping(using Urllib library)
2. Parsing the URL content of the data(using BeautifulSoup library)
3. Data clean-up like removing special characters, numeric values, stop words and punctuations.
4. Tokenization — Creation of tokens (Word tokens and Sentence tokens)
5. Calculate the word frequency for each word.
6. Calculate the weighted frequency for each sentence.
7. Creation of summary choosing 30% of top weighted sentences.


Problem 5
---------

Language detection Using NLTK Python and print the probabilities
and language name for the following phrases:

1. Solen skinner i dag, fuglene synger, og det er sommer.
2. Ní dhéanfaidh ach Dia breithiúnas orm.
3. I domum et cuna matrem tuam in cochleare.
4. Huffa, huffa meg, det finns poteter på badet. Stakkars, stakkars meg, det finns poteter på badet.


Problem 6
---------

Which problems do adaptive and predictive keyboards address?
Explain how touch information and language information can be combined for keyboard adaptation.
Explain decoding of touch sequences with token passing and beam pruning. Using Algorithm

In [1]:
import nltk